In [1]:
!pip install folium shapely openrouteservice geojson geopandas matplotlib

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from shapely import wkt, geometry

import folium
import shapely
import geojson
import json
from pprint import pprint
from openrouteservice import client, directions, geocode
import json
from shapely.geometry import Polygon, MultiPolygon, LineString, shape, GeometryCollection
from shapely.ops import cascaded_union

In [3]:
external = False

if external:
    %store -r
else:
    start_coord = [8.765719, 49.232204]
    api_key = ''
    station_count = 13
    length = 40000
    force_poi_limit = 6000
    force_no_poi_limit = 7000
    roundness = 16
    green = 0.5
    quiet = 0.5
    seed = 10
    grenze = ""

clnt = client.Client(key=api_key, timeout=360)
#clnt = client.Client(base_url='http://localhost:8080/ors', timeout=360)
poi_clnt = client.Client(key=api_key, timeout=360)
#clnt = poi_clnt


In [4]:
with open('./Grenzen/3laender.geojson') as geojson1:
    dreilaender = json.load(geojson1)

with open('./Grenzen/france.geojson') as geojson2:
    france = json.load(geojson2)

with open('./Grenzen/swiss.geojson') as geojson2:
    swiss = json.load(geojson2)

if (grenze == "3laender"):
    borders = dreilaender
elif (grenze == "schweiz"):
    borders = swiss
elif (grenze == "frankreich"):
    borders = france
else:
    borders = swiss

In [5]:


m = folium.Map(tiles='Stamen Toner',location=(start_coord[1], start_coord[0]), zoom_start=14)
folium.Marker(location=[start_coord[1],start_coord[0]], popup="Start").add_to(m)
              
m


In [6]:
query = {
    'coordinates': [start_coord],
    'profile': 'foot-hiking',
    "options":{
        "round_trip":{
            "length":length,
            "points":roundness,
            "seed":seed
        },
        "profile_params":{
            "weightings":{
                "green":green,
                "quiet":quiet
            }
        },
        #"avoid_polygons": borders,
    },
    'format_out': 'geojson',
    'instructions': 'false',
    'units': 'km',
    'radiuses': [200]
}

#print(query)

trip = clnt.directions(**query)
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

folium.features.GeoJson(data=trip,
                        name='Rosscorona',
                        style_function=style_function('#84e184'),
                       overlay=True).add_to(m)

m

ApiError: 401 ({'error': 'Authorization field/api_key missing in request. If you do not have a token, please sign up for one at https://openrouteservice.org'})

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

waypoints = []
for index, waypoint in enumerate(trip['features'][0]['geometry']['coordinates']):
    waypoints.append(Point(waypoint))

waypoints = gpd.GeoDataFrame(geometry = waypoints,crs={'init':'epsg:4326'})
waypoints_projected = waypoints.to_crs(epsg=3310)
waypoints['distance_next'] = waypoints_projected['geometry'].distance(waypoints_projected['geometry'].shift(-1))
waypoints['distance_prev'] = waypoints_projected['geometry'].distance(waypoints_projected['geometry'].shift(1))

waypoints.loc[waypoints['distance_prev'].isna(), 'distance_prev'] = 0
waypoints.loc[waypoints['distance_next'].isna(), 'distance_next'] = 0
waypoints

In [ ]:
from openrouteservice import places


query = {
    'request': 'pois',
    'geojson': trip['features'][0]['geometry'],
    'buffer': 200,
}
poi = poi_clnt.places(**query)

In [ ]:
# https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml
category_ids = [101, 103, 110, 163, 164, 166, 171, 177, 179, 191, 192, 269, 278, 281, 283, 291, 292, 293, 295, 296, 299, 304, 308, 309 , 310, 330, 398, 426, 435, 450, 488, 518, 519, 493, 528, 563, 564, 565, 568, 585, 587, 588, 594, 595, 597, 604, 605, 607, 608, 609, 610, 131, 132, 133, 134, 135, 136, 151, 153, 155, 156, 157, 621, 622, 623, 624, 625, 626, 627 ]
category_220 = list(range(221, 249))
category_ids = category_ids + category_220

selected_pois = []

for feature in poi[0]['features']:
    category = next(iter(feature['properties']['category_ids']))
    if int(category) in category_ids:
        selected_pois.append(feature)


In [ ]:
selected_pois

Takes the complete list of points of interests and narrows them down, so that only "wanted" POI categories are used


In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, MultiPoint
from shapely.ops import nearest_points

rows_list = []

dict1 = {}
dict1["geometry"] = Point(start_coord)
dict1["name"] = "Start"
rows_list.append(dict1)

for index, poi in enumerate(selected_pois):
    dict1 = {}
    dict1["geometry"] = Point(poi['geometry']['coordinates'])
    try:
        dict1["name"] = poi['properties']['osm_tags'].get('name')
    except: 
        dict1["name"]=None
    dict1["category_ids"] = poi['properties']['category_ids'].keys()

    category_names = []
    category_groups =[]
    for category_id in poi['properties']['category_ids'].keys():
        category_names.append(poi['properties']['category_ids'][category_id]['category_name'])
        category_groups.append(poi['properties']['category_ids'][category_id]['category_group'])
    dict1["category_names"] = category_names
    dict1["category_groups"] = category_groups

    rows_list.append(dict1)

poi_points = gpd.GeoDataFrame(rows_list, crs={'init':'epsg:4326'})
poi_points["nearest_waypoint_index"] = 0

indexNames = poi_points[ poi_points['name'].isnull() ].index
poi_points.drop(indexNames , inplace=True)

poi_points



Maps the POIs to the route an saves the nearest waypoint index for each POI

In [ ]:

route = MultiPoint(trip['features'][0]['geometry']['coordinates'])

list_route = list(route)

for i, row in poi_points.iterrows():
    poi_points.at[i,'nearest_waypoint_index'] = int(list_route.index(nearest_points(route, row['geometry'])[0]))

poi_points = poi_points.sort_values(by=['nearest_waypoint_index']).reset_index(drop=True)

poi_points

In [ ]:
poi_points['distance_prev'] = 0
poi_points['distance_next'] = 0

for index, poi in poi_points.iterrows():

    nearest_waypoint_index_current = int(poi['nearest_waypoint_index'])
    try:
        nearest_waypoint_index_next = int(poi_points.loc[index+1]['nearest_waypoint_index'])-1
    except:
        nearest_waypoint_index_next = int(poi_points.loc[index]['nearest_waypoint_index'])-1
        
    poi_points['distance_next'].iloc[index] = sum( waypoints.iloc[nearest_waypoint_index_current : nearest_waypoint_index_next] ['distance_next'])
    try:
        poi_points['distance_prev'].iloc[index+1] = poi_points['distance_next'].iloc[index]
    except:
        pass

poi_points



Calculates the sum of distances from the last and to the next POI for each POI

In [ ]:
poi_points["force_selected"] = False
poi_points.loc[0, "force_selected"] = True

poi_points.loc[(poi_points['distance_prev'] > force_poi_limit) | (poi_points['distance_next'] > force_poi_limit), ["force_selected"]] = True

poi_points[poi_points['force_selected'] == True]


waypoints["pois_selected"] = False
selected_segments = []
blocked_segments = []

last_index = 0
for index, selected_poi in poi_points[poi_points['force_selected'] == True].iterrows():
    waypoint_last_selected_poi = poi_points.iloc[last_index]['nearest_waypoint_index']
    waypoint_current_selected_poi = selected_poi['nearest_waypoint_index']

    # mark sections as selected if two pois are following each other  
    if (last_index+1 == index):
        waypoints.loc[waypoint_last_selected_poi:waypoint_current_selected_poi, 'pois_selected'] = True

        selected_segments.append(list(waypoints.loc[waypoint_last_selected_poi:waypoint_current_selected_poi, 'geometry']))

    # mark sections as selected if the distance between two selected sections is to short 
    else:
        distance = sum(waypoints[waypoint_last_selected_poi:waypoint_current_selected_poi-1]['distance_next'])
        if (distance < force_no_poi_limit):
            waypoints.loc[waypoint_last_selected_poi:waypoint_current_selected_poi, 'pois_selected'] = True
            blocked_segments.append(list(waypoints.loc[waypoint_last_selected_poi:waypoint_current_selected_poi, 'geometry']))


    last_index = index

In [ ]:
poi_points["greedy_selected"] = False

greedy = 0.0

remaining_stations = (station_count+1) - len(poi_points[poi_points['force_selected'] == True])
greedy_distance = sum(waypoints[waypoints['pois_selected']==False]['distance_next'])/remaining_stations
unselected_poi_waypoint_indexes = list(poi_points[poi_points['force_selected']==False]['nearest_waypoint_index'])

if (remaining_stations > 0):

    for index, waypoint in waypoints[waypoints['pois_selected']==False].iterrows():

        if (waypoint['pois_selected']==True):
            greedy = 0

        if (greedy > greedy_distance):
            greedy = 0
            next_poi_index = unselected_poi_waypoint_indexes[next(i for i,v in enumerate(unselected_poi_waypoint_indexes) if v > index)]
            prev_poi_index = unselected_poi_waypoint_indexes[next(i for i,v in enumerate(unselected_poi_waypoint_indexes) if v > index)-1]

            distance_next_poi = sum( waypoints.iloc[index : next_poi_index] ['distance_next'])
            distance_prev_poi = sum( waypoints.iloc[prev_poi_index : index] ['distance_next'])

            if distance_prev_poi < distance_next_poi:
                poi = poi_points[poi_points['nearest_waypoint_index']==prev_poi_index].head(1)
            else:
                poi = poi_points[poi_points['nearest_waypoint_index']==next_poi_index].head(1)

            poi_points.loc[poi.index, 'greedy_selected'] = True

        else:
            greedy += waypoint['distance_next']


poi_points[poi_points['greedy_selected']==True]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
plt.bar(range(0, len(poi_points)), list(poi_points.sort_values(by=['nearest_waypoint_index'])['distance_next'].values))

plt.show()


In [ ]:
if not external:

   poi_map = folium.Map(tiles='Stamen Toner',location=(start_coord[1], start_coord[0]), bbox=trip['features'][0]['bbox'], zoom_start=12)

   folium.features.GeoJson(data=trip,
                           name='Rosscorona',
                           style_function=style_function('#84e184'),
                        overlay=True).add_to(poi_map)

   for index, feature in poi_points.iterrows():
      folium.Circle([feature['geometry'].y, feature['geometry'].x], radius=10, popup=feature['name']).add_to(poi_map)

   for index, feature in poi_points[poi_points['force_selected'] == True].iterrows():
      folium.Marker([feature['geometry'].y, feature['geometry'].x], popup=feature['name']).add_to(poi_map)

   for index, feature in poi_points[poi_points['greedy_selected'] == True].iterrows():
      folium.Marker([feature['geometry'].y, feature['geometry'].x], icon=folium.Icon(color='red'), popup=(str(index) + " " + feature['name']) ).add_to(poi_map)

   for segment in selected_segments:
      line = []
      for point in segment:
         line.append([point.y, point.x])

      folium.PolyLine(line, color='#FF0000' ).add_to(poi_map)

   for segment in blocked_segments:
      line = []
      for point in segment:
         line.append([point.y, point.x])

      folium.PolyLine(line, color='#0000FF' ).add_to(poi_map)


   poi_map
 

In [ ]:
computed_route = []
for point in list(poi_points[(poi_points['greedy_selected'] == True) | (poi_points['force_selected'] == True)]['geometry']):
    computed_route.append([point.x, point.y])
computed_route.append([poi_points.iloc[0]['geometry'].x,poi_points.iloc[0]['geometry'].y])


computed_route

query = {
    'coordinates': computed_route,
    'profile': 'foot-hiking',
    "options":{
        "avoid_borders": "all"
    },
    'format_out': 'geojson',
    'instructions': 'false',
    'units': 'km',
    'elevation': 'true'
}

computed_trip = clnt.directions(**query)



In [ ]:

poi_map = folium.Map(tiles='Stamen Toner',location=(start_coord[1], start_coord[0]), bbox=trip['features'][0]['bbox'], zoom_start=12)

def style_function(color):
    return lambda feature: dict(color=color,
                            weight=3,
                            opacity=1)

folium.features.GeoJson(data=computed_trip,
                        name='Rosscorona',
                        style_function=style_function('#84e184'),
                    overlay=True).add_to(poi_map)

for index, feature in poi_points.iterrows():
    folium.Circle([feature['geometry'].y, feature['geometry'].x], radius=10, popup=feature['name']).add_to(poi_map)

for index, feature in poi_points[poi_points['force_selected'] == True].iterrows():
    folium.Marker([feature['geometry'].y, feature['geometry'].x], popup=feature['name']).add_to(poi_map)

for index, feature in poi_points[poi_points['greedy_selected'] == True].iterrows():
    folium.Marker([feature['geometry'].y, feature['geometry'].x], icon=folium.Icon(color='red'), popup=(str(index) + " " + feature['name']) ).add_to(poi_map)

poi_map

In [ ]:
from shapely.geometry import Point

computed_waypoints = []
elevation = []
for index, waypoint in enumerate(computed_trip['features'][0]['geometry']['coordinates']):
    computed_waypoints.append(Point(waypoint))
    elevation.append(waypoint[2])

computed_waypoints = gpd.GeoDataFrame(geometry = computed_waypoints,crs={'init':'epsg:4326'})
waypoints_projected = computed_waypoints.to_crs(epsg=3310)
computed_waypoints['distance_next'] = waypoints_projected['geometry'].distance(waypoints_projected['geometry'].shift(-1))
computed_waypoints['distance_prev'] = waypoints_projected['geometry'].distance(waypoints_projected['geometry'].shift(1))

computed_waypoints.loc[computed_waypoints['distance_prev'].isna(), 'distance_prev'] = 0
computed_waypoints.loc[computed_waypoints['distance_next'].isna(), 'distance_next'] = 0

poi_waypoints = computed_trip['features'][0]['properties']['way_points']
ascend = computed_trip['features'][0]['properties']['ascent']
distance = computed_trip['features'][0]['properties']['summary']['distance']

computed_waypoints['poi'] = False
distance_per_section = []

last_poi = 0
for index, poi in enumerate(poi_waypoints):
    computed_waypoints.loc[poi, 'poi_number'] = index
    computed_waypoints.loc[poi, 'poi'] = True
    distance_per_section.append(sum(computed_waypoints.iloc[last_poi:poi]['distance_next']))
    last_poi = poi



In [ ]:
import os


result_pois_selected = ""
result_pois = ""
result_route = ""

waypoint_names = poi_points[(poi_points['greedy_selected'] == True) | (poi_points['force_selected'] == True)]['name'].to_list()
waypoint_categories = poi_points[(poi_points['greedy_selected'] == True) | (poi_points['force_selected'] == True)]['category_names'].to_list()


sel = poi_points[(poi_points['greedy_selected'] == True) | (poi_points['force_selected'] == True)]

sel[sel.name != 'Start']['geometry'].to_file("pois_selected.geojson", driver='GeoJSON')
with open('pois_selected.geojson', 'r') as temp:
  result_pois_selected = temp.read()

poi_points['geometry'].to_file("pois.geojson", driver='GeoJSON')
with open('pois.geojson', 'r') as temp:
  result_pois = temp.read()

gpd.GeoSeries(LineString(computed_waypoints['geometry'])).to_file("route.geojson", driver='GeoJSON')
with open('route.geojson', 'r') as temp:
  result_route = temp.read()

os.remove("pois_selected.geojson")
os.remove("pois.geojson")
os.remove("route.geojson")

waypoints = list(zip(waypoint_names, waypoint_categories))
sel[sel.name != 'Start']

In [ ]:
distance

In [ ]:
ascend

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
#plt.plot(poi_points['Distance'].values)
plt.bar(range(0, len(distance_per_section)), distance_per_section)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 8))
plt.plot(elevation)
plt.ylabel('elevation')
plt.show()

In [ ]:
poi_map

In [ ]:
%store -z
%store ascend distance distance_per_section elevation result_pois_selected result_pois result_route waypoints

In [ ]:
waypoints